<a href="https://colab.research.google.com/github/sisomimoctrl/study-pytorch/blob/main/logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(777)

"""
    0: 기본골격
    1: nn.Sequential 구현
    2: class 구현
"""
method = 2
print(f'method : {method}\n')

# 데이터
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

# 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)


# optimizer 설정
if method == 0:
    optimizer = optim.SGD([W, b], lr=1)
elif method == 1:
    model = nn.Sequential(
        nn.Linear(2, 1), # input_dim = 2, output_dim = 1
        nn.Sigmoid() # 출력은 시그모이드 함수를 거친다
    )
    optimizer = optim.SGD(model.parameters(), lr=1)
elif method == 2:
    class BinaryClassifier(nn.Module):
        def __init__(self):
            super().__init__()
            self.linear = nn.Linear(2, 1)
            self.sigmoid = nn.Sigmoid()
            
        def forward(self, x):
            return self.sigmoid(self.linear(x))

    model = BinaryClassifier()
    optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    if method == 0:
        hypothesis = torch.sigmoid(x_train.matmul(W) + b)
        cost = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis)).mean()
    else:
        hypothesis = model(x_train)
        cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f}')
        
# 검증
if method == 0:
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    prediction = hypothesis >= torch.FloatTensor([0.5])
    print(f'\n{prediction}\n')
    print(W,b)
else:
    hypothesis = model(x_train)
    prediction = hypothesis >= torch.FloatTensor([0.5])
    print(f'\n{prediction}\n')   
    print(list(model.parameters()))

method : 2

Epoch    0/1000 Cost: 1.712296
Epoch  100/1000 Cost: 0.133719
Epoch  200/1000 Cost: 0.080292
Epoch  300/1000 Cost: 0.057722
Epoch  400/1000 Cost: 0.045192
Epoch  500/1000 Cost: 0.037188
Epoch  600/1000 Cost: 0.031620
Epoch  700/1000 Cost: 0.027517
Epoch  800/1000 Cost: 0.024364
Epoch  900/1000 Cost: 0.021864
Epoch 1000/1000 Cost: 0.019832

tensor([[False],
        [False],
        [False],
        [ True],
        [ True],
        [ True]])

[Parameter containing:
tensor([[3.2540, 1.5184]], requires_grad=True), Parameter containing:
tensor([-14.4865], requires_grad=True)]
